In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Input, Dense, Dropout, BatchNormalization, Add, Activation, concatenate, Lambda, Flatten
from tensorflow.keras.models import Model
import numpy as np


In [2]:
def multi_scale_conv_block(input_tensor, filters, kernel_size=3, dilation_rates=[1, 2, 4]):
    x = input_tensor
    for dilation_rate in dilation_rates:
        residual = Conv1D(filters, kernel_size, dilation_rate=dilation_rate, padding='same')(x)
        residual = BatchNormalization()(residual)
        residual = Activation('relu')(residual)
        residual = Dropout(0.2)(residual)
        x = Add()([x, residual])
    return x
def periodic_coding(timestamps, max_values):
    sin_features = tf.math.sin(2 * np.pi * timestamps / max_values)
    cos_features = tf.math.cos(2 * np.pi * timestamps / max_values)
    periodic_features = tf.concat([sin_features, cos_features], axis=-1)
    return periodic_features
def fully_connected_sub_network(input_tensor, units):
    x = Dense(units, activation='relu')(input_tensor)
    x = Dropout(0.2)(x)
    x = Dense(units, activation='relu')(x)
    return x
def build_tcms_cnn(input_shape, output_steps, num_filters=24, kernel_size=3, dilation_rates=[1, 2, 4]):
    # Inputs
    inputs = Input(shape=input_shape, name='input_features')
    timestamps = Input(shape=(input_shape[0], 3), name='input_timestamps')  # Assuming the timestamps have three features (e.g., hour, day, month)
    
    # Periodic coding
    max_values = [24, 7, 12]  # Maximum values for hours, days, and months
    periodic_features = Lambda(lambda x: periodic_coding(x, max_values))(timestamps)
    
    # Concatenate periodic features with input
    x = concatenate([inputs, periodic_features], axis=-1)
    
    # Initial convolution layers (convolution 0)
    x = Conv1D(64, kernel_size, padding='same', activation='relu')(x)
    x = Conv1D(num_filters, kernel_size, padding='same', activation='relu')(x)
    
    # Apply multi-scale convolutions (MS-CNN sub-network)
    for _ in range(0, 8):  # Depth of 32
        x = multi_scale_conv_block(x, num_filters, kernel_size, dilation_rates)
    
    # Flatten the MS-CNN output
    x = Flatten()(x)
    
    # Apply fully connected layers to periodic coding for future steps
    periodic_input_future = Input(shape=(output_steps, 3), name='periodic_input_future')
    periodic_features_future = Lambda(lambda x: periodic_coding(x, max_values))(periodic_input_future)
    periodic_features_future = Flatten()(periodic_features_future)
    fully_connected_output = fully_connected_sub_network(periodic_features_future, num_filters)
    
    # Fusion layer
    fused = concatenate([x, fully_connected_output])
    
    # Final fully connected layer to predict future load
    output = Dense(output_steps)(fused)
    
    # Define the model
    model = Model(inputs=[inputs, timestamps, periodic_input_future], outputs=output)
    return model


In [4]:
input_shape = (240, 3)  # 240 time steps and 3 features (e.g., load, temperature, humidity)
output_steps = 60  # Predicting the next 60 time steps

model = build_tcms_cnn(input_shape, output_steps)
model.compile(optimizer='adam', loss='mse')

# Example data
X_train = np.random.rand(1000, 240, 3)
timestamps_train = np.random.randint(0, [24, 7, 12], size=(1000, 240, 3))
timestamps_future = np.random.randint(0, [24, 7, 12], size=(1000, 60, 3))
y_train = np.random.rand(1000, 60)

model.fit([X_train, timestamps_train, timestamps_future], y_train, epochs=10, batch_size=32, verbose=True)


Epoch 1/10
32/32 [==============================] - 10s 28ms/step - loss: 259.9205
Epoch 2/10
32/32 [==============================] - 1s 24ms/step - loss: 26.4140
Epoch 3/10
32/32 [==============================] - 1s 23ms/step - loss: 14.9189
Epoch 4/10
32/32 [==============================] - 1s 23ms/step - loss: 12.8791
Epoch 5/10
32/32 [==============================] - 1s 23ms/step - loss: 10.7845
Epoch 6/10
32/32 [==============================] - 1s 23ms/step - loss: 9.4662
Epoch 7/10
32/32 [==============================] - 1s 23ms/step - loss: 8.5395
Epoch 8/10
32/32 [==============================] - 1s 23ms/step - loss: 7.5806
Epoch 9/10
32/32 [==============================] - 1s 23ms/step - loss: 6.7447
Epoch 10/10
32/32 [==============================] - 1s 23ms/step - loss: 6.2497


In [7]:
model.predict([X_train, timestamps_train, timestamps_future]).shape

32/32 [==============================] - 0s 13ms/step


(1000, 60)

In [11]:
X_train.shape

(1000, 240, 3)

In [9]:
timestamps_future.shape

(1000, 60, 3)

In [10]:
timestamps_future

array([[[21,  6,  0],
        [ 8,  0,  4],
        [20,  2,  7],
        ...,
        [ 4,  4,  1],
        [21,  5,  3],
        [ 3,  1,  3]],

       [[ 5,  1,  9],
        [ 8,  5, 10],
        [13,  4,  0],
        ...,
        [23,  4,  8],
        [20,  6,  4],
        [ 6,  4,  9]],

       [[ 8,  1,  7],
        [ 8,  1,  9],
        [ 8,  2,  4],
        ...,
        [ 8,  0,  9],
        [ 8,  2,  6],
        [ 1,  1,  3]],

       ...,

       [[22,  5,  7],
        [10,  6,  0],
        [12,  3,  5],
        ...,
        [23,  2,  3],
        [ 9,  1,  7],
        [ 3,  6,  9]],

       [[ 7,  4,  6],
        [ 6,  2,  5],
        [ 6,  2,  4],
        ...,
        [20,  0,  3],
        [ 4,  6,  7],
        [12,  6,  1]],

       [[13,  1,  2],
        [17,  2,  2],
        [17,  4,  1],
        ...,
        [ 8,  0,  2],
        [ 6,  2, 11],
        [15,  0,  6]]])